<a href="https://colab.research.google.com/github/shashank1947/NNDL/blob/main/NNDL_Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

In [4]:
data = pd.read_excel('/content/drive/MyDrive/Load Data in kW.xlsx')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [5]:
X = []
Y = []
for i in range(24, data.shape[0]):
    X.append(data['Load (kW)'].iloc[i-25]) # one hour and one day before
    Y.append(data['Load (kW)'].iloc[i])

In [6]:
data = pd.DataFrame({'Prev Day Load' : X, 'Pres Day Load' : Y})
data.head()

,Prev Day Load,Pres Day Load
0,6938.37468,4931.26380
1,5551.82208,4775.53968
2,4983.17184,4713.81120
3,4888.39680,4689.02628
4,5072.95872,4844.75040


In [7]:
Xmax = np.max(data['Prev Day Load'])
Xmin = np.min(data['Prev Day Load'])
Ymax = np.max(data['Pres Day Load'])
Ymin = np.min(data['Pres Day Load'])
data['Prev Day Load'] = (data['Prev Day Load'] - Xmin) / (Xmax - Xmin)
data['Pres Day Load'] = (data['Pres Day Load'] - Ymin) / (Ymax - Ymin)
data.head()

,Prev Day Load,Pres Day Load
0,0.651650,0.284300
1,0.397877,0.255799
2,0.293800,0.244501
3,0.276454,0.239965
4,0.310234,0.268466


In [8]:
x = data['Prev Day Load']
y = data['Pres Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=101)

In [9]:
#Initialization
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.1
epochs = 500
gm = 0.9
Em2 = 0
Ec2 = 0
e = 1E-6

# Loop for epochs
for _ in range(epochs):
    
    for i in range(Xtrain.shape[0]):
        # Calculating Grad M
        gradM = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c) * Xtrain.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c)
        
        # Calculating updated values of Em2 and Ec2
        Em2 = (gm*Em2) + ((1-gm)*(gradM ** 2))
        Ec2 = (gm*Ec2) + ((1-gm)*(gradC ** 2))
        
        # Updating m and c values
        m -= (eta * gradM) / ((e + Em2) ** 0.5)
        c -= (eta * gradC) / ((e + Ec2) ** 0.5)

# Print model parameters after training
print(f'm = {m}\nc = {c}')

m = 0.7459993025267384
c = 0.14754753492225678


In [10]:
train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]

In [11]:
train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
trainY = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
testY = [i * (Ymax - Ymin) + Ymin for i in Ytest]


In [12]:
# Training Error Calculation
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

# Testing Error Calculation
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 631.4166200569463
MSE : 691375.3194148488
RMSE : 831.4898191889356
Testing Error :
MAE : 613.0434535946623
MSE : 662035.694526841
RMSE : 813.655759229197


In [13]:
pd.DataFrame({'Actual Values' : trainY, 'Predicted Values' : train_pred})

,Actual Values,Predicted Values
0,4221.85392,4724.814053
1,7609.43808,6742.382594
2,5847.05880,6601.676085
3,4672.50300,5074.254598
4,7883.78688,6141.182055
...,...,...
1939,5830.37964,5800.230414
1940,6110.49600,6717.497311
1941,5289.94368,6117.343349
1942,7962.97392,7279.625630


In [14]:
pd.DataFrame({'Actual Values' : testY, 'Predicted Values' : test_pred})

,Actual Values,Predicted Values
0,6161.624640,6185.370876
1,5761.792440,6040.245485
2,7390.270800,6496.088060
3,4597.368840,6404.919545
4,5242.712040,5936.052896
...,...,...
211,6764.319072,7340.792261
212,7390.270800,6588.884584
213,6550.077600,6512.135579
214,5722.043040,5488.815512


In [15]:
x = float(input('Enter the load at present hour on previouus day : '))
x = (x - Xmin) / (Xmax - Xmin)
prediction = m * x + c
prediction = (prediction * (Ymax - Ymin)) + Ymin
print('Predicted load at present hour :', prediction)

Enter the load at present hour on previouus day : 2600
Predicted load at present hour : 3603.7549472038972
